In [1]:
import pandas as pd

# Домашнее задание к лекции "Базовые понятия статистики"

Будем осуществлять работу с непростым [набором данных](https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.csv) о состоянии здоровья лошадей, испытывающих кишечные колики. Цель – максимально корректно заполнить пропуски.

### Задание 1. Загрузка данных

Изучить представленный набор данных на основе [описания его столбцов](https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.names), загрузить его и оставить 8 столбцов для дальнейшего изучения: `surgery?`, `Age`, `rectal temperature`, `pulse`, `respiratory rate`, `temperature of extremities`, `pain`, `outcome`. 

### Задание 2. Первичное изучение данных

Проанализировать значения по столбцам, рассчитать базовые статистики, найти выбросы.


### Задание 3. Работа с пропусками

Рассчитать количество пропусков для всех выбранных столбцов. Принять и обосновать решение о методе заполнения пропусков по каждому столбцу на основе рассчитанных статистик и возможной взаимосвязи значений в них. Сформировать датафрейм, в котором пропуски будут отсутствовать.

In [2]:
# Задание1
horse = pd.read_csv('./hw_1/horse_data.csv', na_values='?') # загружаю данные, сразу заменяем пустые значения
# > загружаю названия колонок
names_str = ('surgery, Age, Hospital_Number, rectal_temperature, pulse, respiratory_rate, temperature_of_extremities, peripheral_pulse, mucous_membranes, capillary_refill_time, pain, peristalsis, abdominal_distension, nasogastric_tube, nasogastric_reflux, nasogastric_reflux_PH, rectal_examination_feces, abdomen, packed_cell_volume, total_protein, abdominocentesis_appearance, abdomcentesis_total_protein, outcome, surgical_lesion?, type_of_lesion_1, type_of_lesion_2, type_of_lesion_3, cp_data')
horse.columns = names_str.split(sep=', ')
# удаляю лишние колонки
horse.drop(horse.columns.difference(['surgery', 'Age', 'rectal_temperature', 'pulse', 'respiratory_rate', 'temperature_of_extremities', 'pain', 'outcome']), axis=1, inplace=True)
horse.head(1)

,surgery,Age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
0,1.0,1,39.2,88.0,20.0,NaN,3.0,3.0


In [3]:
# Задание 2
horse.info() # смотрю информацию по датафрейму

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   surgery                     298 non-null    float64
 1   Age                         299 non-null    int64  
 2   rectal_temperature          239 non-null    float64
 3   pulse                       275 non-null    float64
 4   respiratory_rate            241 non-null    float64
 5   temperature_of_extremities  243 non-null    float64
 6   pain                        244 non-null    float64
 7   outcome                     298 non-null    float64
dtypes: float64(7), int64(1)
memory usage: 18.8 KB


In [4]:
horse.median()

surgery                        1.0
Age                            1.0
rectal_temperature            38.2
pulse                         64.0
respiratory_rate              24.0
temperature_of_extremities     3.0
pain                           3.0
outcome                        1.0
dtype: float64

In [5]:
horse.describe() # смотрю основные статистики
# на первый взгляд, никаких явно выраженных отклонений нет:
# surgery - не было у большинства лошадей
# Age - странным кажется, что лошади в основном молодые, почти жеребята, возраст чуть больше года
# rectal_temperature - при норме в 37.8 средняя температура чуть повышенная 38.2. Выбиваются min и max значения. На первый взгляд, максимальное значение не выглядит
    # неестественным, скорее всего, оно было зафиксировано у старой лошади, которая впоследствии умерла. Больше вопросов вызывает минимальное значение, но сделать какие-либо
    # выводы на данном этапе сложно.
# respiratory_rate - похоже, что коррелируется с температурой, комментарии те же 
# temperature_of_extremities  - субъективные значения, ничего странного в них не заметно
# pain - субъективные значения, ничего странного в них не заметно
# outcome - т.к. п. 2 и 3 свидетельствуют о гибели лошади, то среднее в 1.5 позволяеь предположить, что большинство лошадей выжил

,surgery,Age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
count,298.000000,299.00000,239.000000,275.000000,241.000000,243.000000,244.000000,298.000000
mean,1.395973,1.64214,38.166527,71.934545,30.427386,2.345679,2.942623,1.550336
std,0.489881,2.17730,0.733508,28.680522,17.678256,1.046369,1.303993,0.737967
min,1.000000,1.00000,35.400000,30.000000,8.000000,1.000000,1.000000,1.000000
25%,1.000000,1.00000,37.800000,48.000000,18.000000,1.000000,2.000000,1.000000
50%,1.000000,1.00000,38.200000,64.000000,24.000000,3.000000,3.000000,1.000000
75%,2.000000,1.00000,38.500000,88.000000,36.000000,3.000000,4.000000,2.000000
max,2.000000,9.00000,40.800000,184.000000,96.000000,4.000000,5.000000,3.000000


In [6]:
# далее начинаю смотреть выбросы

In [7]:
horse.pivot_table(index='Age', values='outcome', aggfunc='count')

,outcome
Age,
1,274
9,24


In [8]:
# Возраст - в выбросы попадают лошади 9-лет, для лошади это не очень большой возраст, но, возможно, это выбросы применительно к данной выборке. В ячейке выше смотрю
# сводную таблицу и смотрю сколько лошадей какого возраста. Как видно, лошади всего 2-х возрастов 1 и 9 лет, при этом 1-легних более 90%, соответственно предположу,
# что две эти группы нужно рассматривать отдельно. Применительно к данной выборке считаем, что 9-лошади являются выбросами, т.к. клиническая картина у них может отличаться от
# молодых лошадей. Отсекаю выбросы.
q1 = horse['Age'].quantile(0.25)
q3 = horse['Age'].quantile(0.75)
iqr = q3 - q1
l_bound = q1 - (1.5 * iqr) 
h_bound = q3 + (1.5 * iqr)
# horse[~horse['Age'].between(l_bound, h_bound, inclusive='both').dropna()] # посмотреть без пустых значений
horse_out_age = horse[horse['Age'].between(l_bound, h_bound, inclusive='both')]
horse_out_age

,surgery,Age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
0,1.0,1,39.2,88.0,20.0,NaN,3.0,3.0
1,2.0,1,38.3,40.0,24.0,1.0,3.0,1.0
3,2.0,1,37.3,104.0,35.0,NaN,NaN,2.0
4,2.0,1,NaN,NaN,NaN,2.0,2.0,1.0
5,1.0,1,37.9,48.0,16.0,1.0,3.0,1.0
...,...,...,...,...,...,...,...,...
294,1.0,1,NaN,120.0,70.0,4.0,2.0,3.0
295,2.0,1,37.2,72.0,24.0,3.0,4.0,3.0
296,1.0,1,37.5,72.0,30.0,4.0,4.0,2.0
297,1.0,1,36.5,100.0,24.0,3.0,3.0,1.0


In [9]:
# по температуре, на мой взгляд, случаи повышенной температуры не являются выбросами, тем более, что прослеживается связь между повышенной температурой и смертью лошади.
# но т.к. задание устранить выбросы, то эти тоже выкидываю
q1 = horse_out_age['rectal_temperature'].quantile(0.25)
q3 = horse_out_age['rectal_temperature'].quantile(0.75)
iqr = q3 - q1
l_bound = q1 - (1.5 * iqr) 
h_bound = q3 + (1.5 * iqr)
# horse_out_age[~horse_out_age['rectal_temperature'].between(l_bound, h_bound, inclusive='both')].dropna() # посмотреть без пустых значений
# далее отсекаю выбросы, но оставляю пустые значения
horse_out_temp = horse_out_age[(horse_out_age['rectal_temperature'].between(l_bound, h_bound, inclusive='both'))|(horse_out_age['rectal_temperature'].isna())]
horse_out_temp

,surgery,Age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
0,1.0,1,39.2,88.0,20.0,NaN,3.0,3.0
1,2.0,1,38.3,40.0,24.0,1.0,3.0,1.0
3,2.0,1,37.3,104.0,35.0,NaN,NaN,2.0
4,2.0,1,NaN,NaN,NaN,2.0,2.0,1.0
5,1.0,1,37.9,48.0,16.0,1.0,3.0,1.0
...,...,...,...,...,...,...,...,...
293,2.0,1,38.5,40.0,16.0,1.0,2.0,1.0
294,1.0,1,NaN,120.0,70.0,4.0,2.0,3.0
295,2.0,1,37.2,72.0,24.0,3.0,4.0,3.0
296,1.0,1,37.5,72.0,30.0,4.0,4.0,2.0


In [10]:
# пульс находит только 2 слишком высоких значения, их можно считать выбросами, хотя опять же, данные явно не ошибочные
# отсеялись они сами вместе с выбросами температуры
q1 = horse_out_temp['pulse'].quantile(0.25)
q3 = horse_out_temp['pulse'].quantile(0.75)
iqr = q3 - q1
l_bound = q1 - (1.5 * iqr) 
h_bound = q3 + (1.5 * iqr)
# horse_out_temp[~horse_out_temp['pulse'].between(l_bound, h_bound, inclusive='both')].dropna() # посмотреть без пустых значений
# horse_outliers = horse_out_temp[horse_out_temp['pulse'].between(l_bound, h_bound, inclusive='both')].dropna()
horse_out_pulse = horse_out_temp[(horse_out_temp['pulse'].between(l_bound, h_bound, inclusive='both'))|(horse_out_temp['pulse'].isna())]
horse_out_pulse

,surgery,Age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
0,1.0,1,39.2,88.0,20.0,NaN,3.0,3.0
1,2.0,1,38.3,40.0,24.0,1.0,3.0,1.0
3,2.0,1,37.3,104.0,35.0,NaN,NaN,2.0
4,2.0,1,NaN,NaN,NaN,2.0,2.0,1.0
5,1.0,1,37.9,48.0,16.0,1.0,3.0,1.0
...,...,...,...,...,...,...,...,...
293,2.0,1,38.5,40.0,16.0,1.0,2.0,1.0
294,1.0,1,NaN,120.0,70.0,4.0,2.0,3.0
295,2.0,1,37.2,72.0,24.0,3.0,4.0,3.0
296,1.0,1,37.5,72.0,30.0,4.0,4.0,2.0


In [11]:
horse.describe()

,surgery,Age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
count,298.000000,299.00000,239.000000,275.000000,241.000000,243.000000,244.000000,298.000000
mean,1.395973,1.64214,38.166527,71.934545,30.427386,2.345679,2.942623,1.550336
std,0.489881,2.17730,0.733508,28.680522,17.678256,1.046369,1.303993,0.737967
min,1.000000,1.00000,35.400000,30.000000,8.000000,1.000000,1.000000,1.000000
25%,1.000000,1.00000,37.800000,48.000000,18.000000,1.000000,2.000000,1.000000
50%,1.000000,1.00000,38.200000,64.000000,24.000000,3.000000,3.000000,1.000000
75%,2.000000,1.00000,38.500000,88.000000,36.000000,3.000000,4.000000,2.000000
max,2.000000,9.00000,40.800000,184.000000,96.000000,4.000000,5.000000,3.000000


In [12]:
# аналогично отсекаю частоту дыхания. стандартный подход здесь не работает, т.к. нижняя граница уходит ниже 0, поэтому за нижнюю границу беру минимум + 25%
q1 = horse_out_pulse['respiratory_rate'].quantile(0.25)
q3 = horse_out_pulse['respiratory_rate'].quantile(0.75)
iqr = q3 - q1
min_ = horse_out_pulse['respiratory_rate'].min()
l_bound = min_*1.25
h_bound = q3 + (1.5 * iqr)
horse_out_pulse[horse_out_pulse['respiratory_rate'].between(l_bound, h_bound, inclusive='both')].dropna() # посмотреть без пустых значений
# horse_out_pulse[horse_out_pulse['respiratory_rate'] > h_bound] # смотрю сколько выше верхней отсечки
# horse_out_pulse[horse_out_pulse['respiratory_rate'] < l_bound] # смотрю сколько ниже нижней отсечки
# horse_outliers = horse_out_temp[horse_out_temp['pulse'].between(l_bound, h_bound, inclusive='both')].dropna()
horse_out_fin = horse_out_pulse[(horse_out_pulse['respiratory_rate'].between(l_bound, h_bound, inclusive='both'))|(horse_out_pulse['respiratory_rate'].isna())]
horse_out_fin

,surgery,Age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
0,1.0,1,39.2,88.0,20.0,NaN,3.0,3.0
1,2.0,1,38.3,40.0,24.0,1.0,3.0,1.0
3,2.0,1,37.3,104.0,35.0,NaN,NaN,2.0
4,2.0,1,NaN,NaN,NaN,2.0,2.0,1.0
5,1.0,1,37.9,48.0,16.0,1.0,3.0,1.0
...,...,...,...,...,...,...,...,...
292,1.0,1,NaN,78.0,24.0,3.0,NaN,3.0
293,2.0,1,38.5,40.0,16.0,1.0,2.0,1.0
295,2.0,1,37.2,72.0,24.0,3.0,4.0,3.0
296,1.0,1,37.5,72.0,30.0,4.0,4.0,2.0


In [13]:
# получаю финальны датафрейм, из которого отсечены все выбросы. строго говоря 47 отсечённых строк на мой взгляд многовато и на реальных рабочих данных я бы определил
# выбросы как min + 5% и max - 5% для каждого параметра соответственно

In [14]:
# задание 3
horse_out_fin.isna().sum() # считаю количество пропущенных значений для каждого из столбцов в штуках

surgery                        1
Age                            0
rectal_temperature            54
pulse                         22
respiratory_rate              52
temperature_of_extremities    45
pain                          43
outcome                        1
dtype: int64

In [15]:
# сначала определяюсь с surgery и outcome, тем более, что это одна строка. для этого вручную смотрю лошадей со схожими показателями температуры, пульса и дыхания
# horse_out_fin[horse_out_fin['rectal_temperature'].between(37.8, 38.5) & horse_out_fin['pulse'].between(40, 58)]
horse_out_fin[horse_out_fin['surgery'].isna() | horse_out_fin['outcome'].isna()]

,surgery,Age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
131,NaN,1,38.0,48.0,20.0,3.0,4.0,NaN


In [16]:
# как видно, большинсву лошадей со схожими показателями делали операцию и все они выжили
horse_out_fin[horse_out_fin['rectal_temperature'].between(37.8, 38.2) & horse_out_fin['pulse'].between(48, 48) & horse_out_fin['respiratory_rate'].between(10, 30)]

,surgery,Age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
5,1.0,1,37.9,48.0,16.0,1.0,3.0,1.0
26,1.0,1,37.8,48.0,12.0,3.0,NaN,1.0
68,2.0,1,38.0,48.0,12.0,1.0,1.0,1.0
70,1.0,1,37.8,48.0,28.0,1.0,1.0,1.0
131,NaN,1,38.0,48.0,20.0,3.0,4.0,NaN
142,1.0,1,37.8,48.0,28.0,1.0,1.0,1.0
154,2.0,1,38.0,48.0,20.0,3.0,1.0,1.0
180,2.0,1,37.8,48.0,14.0,NaN,3.0,1.0
191,1.0,1,38.2,48.0,18.0,1.0,3.0,1.0


In [17]:
# заполняю пропуски для surgery и outcome
# температура средняя и медиана очень близки, поэтому заполню пропуски средней температурой
# temperature_of_extremities и pain субъективные показатели, ни на что особо не влияют, заполняю их по моде
horse_out_fill = horse_out_fin.copy()
horse_out_fill['surgery'] = horse_out_fill['surgery'].fillna('1').astype('int64')
horse_out_fill['outcome'] = horse_out_fill['outcome'].fillna('1').astype('int64')
horse_out_fill['rectal_temperature'] = horse_out_fill['rectal_temperature'].fillna(horse_out_fill['rectal_temperature'].mean()).round(1)
horse_out_fill['temperature_of_extremities'] = horse_out_fill['temperature_of_extremities'].fillna(horse_out_fill['temperature_of_extremities'].mode()[0])
horse_out_fill['pain'] = horse_out_fill['pain'].fillna(horse_out_fill['pain'].mode()[0])
horse_out_fill.isna().sum()

surgery                        0
Age                            0
rectal_temperature             0
pulse                         22
respiratory_rate              52
temperature_of_extremities     0
pain                           0
outcome                        0
dtype: int64

In [18]:
horse_out_fill.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 252 entries, 0 to 298
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   surgery                     252 non-null    int64  
 1   Age                         252 non-null    int64  
 2   rectal_temperature          252 non-null    float64
 3   pulse                       230 non-null    float64
 4   respiratory_rate            200 non-null    float64
 5   temperature_of_extremities  252 non-null    float64
 6   pain                        252 non-null    float64
 7   outcome                     252 non-null    int64  
dtypes: float64(5), int64(3)
memory usage: 17.7 KB


In [19]:
# в процессе изучения датафрейма я заметил, что пульс и дыхание зависят от показателей пар температура-дыхание и температура-пульс соответственно, поэтому:
# сначала пытался заполнять их по среднему значению этих пар соответственно
# horse_out_fill['respiratory_rate'] = horse_out_fill['respiratory_rate'].fillna(horse_out_fill.groupby(['rectal_temperature','pulse'])['respiratory_rate'].transform('mean'))

# но т.к. значение pulse в некоторых строках пустое, то на выходе также получаются пустоты, поэтому
# пульс заполняю в зависимости от температуры
horse_out_fill['pulse'] = horse_out_fill['pulse'].fillna(horse_out_fill.groupby(['rectal_temperature'])['pulse'].transform('mean')).round(0)

# далее заполняю дыхание на основе пары температура-пульс 
horse_out_fill['respiratory_rate'] = horse_out_fill['respiratory_rate'].fillna(horse_out_fill.groupby(['rectal_temperature','pulse'])['respiratory_rate'].transform('mean'))

# а для тех строк, где пары не нашлось, заполняю на основе температуры
horse_out_fill['respiratory_rate'] = horse_out_fill['respiratory_rate'].fillna(horse_out_fill.groupby(['rectal_temperature'])['respiratory_rate'].transform('mean'))

horse_out_fill


,surgery,Age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
0,1,1,39.2,88.0,20.0,3.0,3.0,3
1,2,1,38.3,40.0,24.0,1.0,3.0,1
3,2,1,37.3,104.0,35.0,3.0,3.0,2
4,2,1,38.1,76.0,24.0,2.0,2.0,1
5,1,1,37.9,48.0,16.0,1.0,3.0,1
...,...,...,...,...,...,...,...,...
292,1,1,38.1,78.0,24.0,3.0,3.0,3
293,2,1,38.5,40.0,16.0,1.0,2.0,1
295,2,1,37.2,72.0,24.0,3.0,4.0,3
296,1,1,37.5,72.0,30.0,4.0,4.0,2


In [20]:
# на выходе получаю датафрейм без пропущенных значений
horse_out_fill.isna().sum()

surgery                       0
Age                           0
rectal_temperature            0
pulse                         0
respiratory_rate              0
temperature_of_extremities    0
pain                          0
outcome                       0
dtype: int64

In [21]:
horse.describe() # для сравнения

,surgery,Age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
count,298.000000,299.00000,239.000000,275.000000,241.000000,243.000000,244.000000,298.000000
mean,1.395973,1.64214,38.166527,71.934545,30.427386,2.345679,2.942623,1.550336
std,0.489881,2.17730,0.733508,28.680522,17.678256,1.046369,1.303993,0.737967
min,1.000000,1.00000,35.400000,30.000000,8.000000,1.000000,1.000000,1.000000
25%,1.000000,1.00000,37.800000,48.000000,18.000000,1.000000,2.000000,1.000000
50%,1.000000,1.00000,38.200000,64.000000,24.000000,3.000000,3.000000,1.000000
75%,2.000000,1.00000,38.500000,88.000000,36.000000,3.000000,4.000000,2.000000
max,2.000000,9.00000,40.800000,184.000000,96.000000,4.000000,5.000000,3.000000


In [22]:
horse_out_fill.describe() # для сравнения 

,surgery,Age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
count,252.000000,252.0,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000
mean,1.416667,1.0,38.118254,67.535714,25.986799,2.464286,2.948413,1.531746
std,0.493988,0.0,0.506437,22.651938,10.052023,0.983284,1.214667,0.738109
min,1.000000,1.0,36.600000,30.000000,10.000000,1.000000,1.000000,1.000000
25%,1.000000,1.0,37.875000,48.000000,20.000000,1.000000,2.000000,1.000000
50%,1.000000,1.0,38.100000,60.000000,24.000000,3.000000,3.000000,1.000000
75%,2.000000,1.0,38.400000,80.500000,32.000000,3.000000,4.000000,2.000000
max,2.000000,1.0,39.600000,132.000000,60.000000,4.000000,5.000000,3.000000
